## Part IV:  Features and Labeling Functions

## Generating Features

In [1]:
# # If necessary:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ labels snorkel.db');

In [2]:
# %load_ext autoreload
# %autoreload 2

# from snorkel import SnorkelSession
# session = SnorkelSession()

In [3]:
# from snorkel.models import candidate_subclass

# Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

### Loading the `CandidateSet`

In [4]:
# from snorkel.models import CandidateSet
# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()
# dev = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Development Candidates').one()

### Creating feature matrix

In [5]:
# from snorkel.annotations import FeatureManager

# feature_manager = FeatureManager()

In [6]:
# %time F_train = feature_manager.create(session, train, 'Train Features')

In [7]:
# %time F_dev = feature_manager.update(session, dev, 'Train Features', expand_key_set=False)

### Reloading feature matrix

In [8]:
# %time F_train = feature_manager.load(session, train, 'Train Features')

In [9]:
# F_train

In [10]:
# F_train.get_candidate(0)

In [11]:
# F_train.get_key(0)

In [12]:
# from snorkel.utils import get_keys_by_candidate

# for f in get_keys_by_candidate(F_train, F_train.get_candidate(1))[:10]: print f

### Saving feature matrices

In [13]:
# # If necessary:
# import os
# os.system('cp snorkel.db snorkel.db\ featurized');

## Applying LFs

In [14]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ featurized snorkel.db');

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()

In [15]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

### Defining Labeling Functions

In [16]:
from snorkel.lf_helpers import *

LFs = []

# gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# def LF_training_gold(c):
#     training_gold_dict = get_gold_dict(gold_file, 'stg_temp_min')
#     return 1 if ((c[0].parent.document.name).upper(), 
#                  (c[0].get_span()).upper(), 
#                  (''.join(c[1].get_span().split())).upper()) in training_gold_dict else 0
# LFs.append(LF_training_gold)

# from hardware_utils import load_extended_parts_dict
# gold_file ='data/hardware/hardware_gold.csv'
# parts_dict = load_extended_parts_dict(gold_file)
# print "Loaded %d part numbers." % len(parts_dict)

# def LF_parts_dict(c):
#     return 1 if (c.temp.parent.words)[0] in parts_dict else -1
# LFs.append(LF_foo)

# POSITIVE

def LF_to_right(c):
    return 1 if 'to' in get_right_ngrams(c.temp, window=1, n_max=1) else 0
LFs.append(LF_to_right)

def LF_150_right(c):
    return 1 if '150' in get_right_ngrams(c.temp, window=4, n_max=1) else 0
LFs.append(LF_150_right)

def LF_storage_row(c):
    return 1 if 'storage' in get_row_ngrams(c.temp, n_max=2) else 0
LFs.append(LF_storage_row)

def LF_operating_row(c):
    return 1 if 'operating' in get_row_ngrams(c.temp, n_max=2) else 0
LFs.append(LF_operating_row)

def LF_c_row(c):
    return 1 if '%C' in get_row_ngrams(c.temp, n_min=1, n_max=2, case_sensitive=True) else 0
LFs.append(LF_c_row)

def LF_temperature_row(c):
    return 1 if 'temperature' in get_row_ngrams(c.temp) else 0
LFs.append(LF_temperature_row)

def LF_tstg_row(c):
    row_ngrams = get_row_ngrams(c.temp, n_max=1)
    return 1 if ('tstg' in row_ngrams or 'ts' in row_ngrams) else 0
LFs.append(LF_tstg_row)


# NEGATIVE

def LF_temp_outside_table(c):
    return -1 if c.temp.parent.table is None else 0
LFs.append(LF_temp_outside_table)

def LF_vsomething_row(c):
    return -1 if 'V%' in get_row_ngrams(c.temp, infer=True) else 0
LFs.append(LF_vsomething_row)

def LF_collector_row(c):
    return -1 if 'collector' in get_row_ngrams(c.temp) else 0
LFs.append(LF_collector_row)

def LF_collectorattrib_row(c):
    return -1 if 'collector%' in get_row_ngrams(c.temp) else 0
LFs.append(LF_collectorattrib_row)

def LF_typ_row(c):
    return -1 if 'typ' in get_row_ngrams(c.temp) else 0
LFs.append(LF_collector_row)

def LF_complement_left(c):
    return -1 if 'complement' in get_left_ngrams(c.part) else 0
LFs.append(LF_complement_left)

def LF_voltage_row(c):
    return -1 if 'voltage' in get_row_ngrams(c.temp, infer=True) else 0
LFs.append(LF_voltage_row)

def LF_v_row(c):
    return -1 if 'v' in get_row_ngrams(c.temp, infer=True) else 0
LFs.append(LF_v_row)

def LF_vceo_row(c):
    return -1 if 'vceo' in get_row_ngrams(c.temp, infer=True) else 0
LFs.append(LF_vceo_row)

def LF_mindot_row(c):
    return -1 if 'min.' in get_col_ngrams(c.temp) else 0
LFs.append(LF_mindot_row)

def LF_min_row(c):
    return -1 if 'min' in get_col_ngrams(c.temp) else 0
LFs.append(LF_min_row)

def LF_test_row(c):
    return -1 if 'test' in get_row_ngrams(c.temp, infer=True) else 0
LFs.append(LF_test_row)

def LF_condition_row(c):
    return -1 if 'condition' in get_row_ngrams(c.temp, infer=True) else 0
LFs.append(LF_condition_row)

### Testing LFs and LF_helpers

In [17]:
from snorkel.lf_helpers import get_right_ngrams, get_left_ngrams
from snorkel.lf_helpers import contains_token, contains_regex
from snorkel.lf_helpers import get_phrase_ngrams, get_cell_ngrams, get_neighbor_cell_ngrams
from snorkel.lf_helpers import get_row_ngrams, get_col_ngrams, get_aligned_ngrams
from snorkel.lf_helpers import same_document, same_table, same_cell, same_phrase
c = train[12]
print c.part
print c.temp
print c.temp.parent
print c.temp.parent.table
print same_document(c)
print same_table(c)
print contains_regex(c, rgx=r'849')
print LF_temp_outside_table(c)

Span("BC182", parent=84426, chars=[0,4], words=[0,0])
ImplicitSpan("-55", parent=84276, words=[0,0], position=[0])
Phrase(Doc: BC182-D, Table: 0, Row: 7, Col: 2, Position: 0, Text: -55 to +150)
Table(Doc: BC182-D, Position: 0)
True
False
False
0


In [18]:
%time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)
L_train

[========================================] 100%

Loading sparse Label matrix...
CPU times: user 14min 17s, sys: 28.2 s, total: 14min 45s
Wall time: 14min 24s


<75908x19 sparse matrix of type '<type 'numpy.float64'>'
	with 80892 stored elements in Compressed Sparse Row format>

In [19]:
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Gold').one()

In [20]:
# L_train.lf_stats()
%time L_train.lf_stats(train_gold)

[========================================] 100%

CPU times: user 11.7 s, sys: 1.52 s, total: 13.2 s
Wall time: 11.2 s


,accuracy,conflicts,coverage,j,overlaps
LF_to_right,1.000000,0.001278,0.101649,0,0.101649
LF_150_right,1.000000,0.000000,0.107551,1,0.107551
LF_storage_row,1.000000,0.014623,0.258906,2,0.258906
LF_operating_row,1.000000,0.001620,0.064736,3,0.064736
LF_c_row,NaN,0.000000,0.000000,4,0.000000
LF_temperature_row,1.000000,0.027942,0.301431,5,0.274661
LF_tstg_row,1.000000,0.001252,0.024134,6,0.024134
LF_temp_outside_table,0.000000,0.000026,0.019563,7,0.000026
LF_vsomething_row,NaN,0.000000,0.000000,8,0.000000
LF_collector_row,NaN,0.000000,0.000000,9,0.000000


In [21]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ features');

Next, in Part 5, we will test our model on the development `CandidateSet`.

# TEMPORARY

In [22]:
# from snorkel.models import Corpus
# from snorkel.utils import get_ORM_instance
# corpus = get_ORM_instance(Corpus, session, 'Hardware')

# for d in corpus:
#     if d.name == 'PHGLS18216-1':
#         doc = d
# print doc

In [23]:
# for phrase in doc.phrases:
#     for ngram in tokens_to_ngrams(phrase.words, n_max=2, n_min=2):
#         if ngram == '- C':
#             import pdb; pdb.set_trace()